<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/Optimizaci%C3%B3nHiperpar%C3%A1metros/GridSearch_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_validate, KFold
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,make_scorer
import numpy as np
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error

# Cargar datos
dataset_path = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset1CodificadoV2.csv'
data = pd.read_csv(dataset_path)

# Preparar los datos
X = data.drop(columns=['Inputs', 'Unnamed: 0'])
y = data['Inputs']

# Configurar el modelo XGBoost
model = XGBRegressor(objective='reg:squarederror')

# Configurar la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Definir las métricas
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scoring = {
    'rmse': make_scorer(rmse, greater_is_better=False),
    'r2': 'r2',
    'mae': 'neg_mean_absolute_error'
}

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.7, 1],
    'colsample_bytree': [0.7, 1],
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring=scoring, refit='rmse', return_train_score=True, n_jobs=-1, verbose=3)

# Ejecutar Grid Search
grid_search.fit(X, y)

# Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", grid_search.best_params_)

# Mostrar las métricas para la mejor configuración
best_index = grid_search.best_index_
print("Métricas para la mejor configuración:")
for scorer in scoring:
    scorer_name = f"mean_test_{scorer}"
    print(f"{scorer.upper()}: {grid_search.cv_results_[scorer_name][best_index]}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Mejores hiperparámetros: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 100, 'subsample': 1}
Métricas para la mejor configuración:
RMSE: -20.86135997606383
R2: 0.5570783037656029
MAE: -12.2690813968007
